<a href="https://colab.research.google.com/github/RachelRamirez/misclassification_matrix/blob/main/PA_w%5B7%2C9%5D_100_Misclassification_Cost_Matrix_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## What happens when I use PA method with Weighted_Matrix with w_array[7, 9] = 100

The results of running the  ISantaro appeared counterintuitive, because when I increased the cost of a misclassification, more misclassifications were made. It was hard to see at first because it wasn't consistently happening, it happened 7 out of 30 times, but when it happened it was a very large number of misclassifications.   So now I'm looking to compare another code implementation,  by Phil Alton here:  https://stackoverflow.com/a/61963004 

Remember to change the [Admin File stuff]  below and the Weight Matrix before Running

### Reproducible Seeds

In [1]:
#For Reproducibility
import numpy as np
# np.random.seed(1337)  # for reproducibility

import tensorflow as tf
# tf.random.set_seed(33)

import random as python_random
# python_random.seed(4)

# https://www.tensorflow.org/api_docs/python/tf/keras/utils/set_random_seed
tf.keras.utils.set_random_seed(342) #Possibly use next iteration if the above doesn't work


# Running more than once causes variation.  try adding this:
# Set seed value
seed_value = 56
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

print("TF version: " , tf.__version__ )
print("Keras version: " , tf.keras.__version__ )

TF version:  2.11.0
Keras version:  2.11.0


### Import rest of Library

In [2]:

# from https://github.com/keras-team/keras/issues/2115#issuecomment-204060456
# witha correction on the weighted function in the middle 

'''Train a simple deep NN on the MNIST dataset.
Get to 98.40% test accuracy after 20 epochs
(there is *a lot* of margin for parameter tuning).
2 seconds per epoch on a K520 GPU.
'''

from __future__ import print_function  #do i still need this?
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils
import keras.backend as K
from itertools import product
import functools
from functools import partial
from time import ctime
from time import sleep
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd 
from sklearn.metrics import confusion_matrix

## MORE REPEATABILITY STUFF NEEDED - If theres a way to update this to V2 of Tensorflow great, otherwise I had to use TF 1.0 code
# 5. Configure a new global `tensorflow` session (https://stackoverflow.com/questions/50659482/why-cant-i-get-reproducible-results-in-keras-even-though-i-set-the-random-seeds)
# from keras import backend as K


#I believe thecode below is to help things be repeatable each time different sections in my google colab notebook execute
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
K.set_session(sess)

### Define batch, epochs, and format data

In [3]:
batch_size = 256 # I originally had it very  high batch size to reduce the variation in the data each batch and hope it makes the model training more nearly identical which it did, then i bring it back down to something reasonable to get better results training the NN
nb_classes = 10
nb_epoch = 15

# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()


X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)


60000 train samples
10000 test samples


### Define Same Model but use normal Categorical CrossEntropy with no extra cost-matrix of Weights

In [4]:
# def normal_method():

#   model = Sequential()
#   model.add(Dense(512, input_shape=(784,) ,kernel_initializer=tf.keras.initializers.glorot_uniform(seed=42)))
#   model.add(Activation('relu'))
#   model.add(Dropout(0.2))
#   model.add(Dense(512, kernel_initializer=tf.keras.initializers.glorot_uniform(seed=42)))
#   model.add(Activation('relu'))
#   model.add(Dropout(0.2))
#   model.add(Dense(10, kernel_initializer=tf.keras.initializers.glorot_uniform(seed=42)))
#   model.add(Activation('softmax'))

#   rms = RMSprop()
#   # model.compile(loss=ncce, optimizer=rms)
#   model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=rms, metrics='categorical_accuracy', )

#   #add early_stop to prevent overfittings
#   # callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

#   model_history = model.fit(X_train, Y_train,
#             batch_size=batch_size, epochs=nb_epoch, verbose=2,
#             validation_data=(X_test, Y_test),shuffle=False, use_multiprocessing=True
#             , callbacks = [callback])

  
#   # model.evaluate(X_test, Y_test, verbose=1)  # I know this isn't the typical use of train/val/test sets, please dont' comment on that
  
#   #Predict
#   y_prediction = model.predict(X_test)
#   y_prediction  = np.argmax(y_prediction, axis=1)
#   # Y_prediction = np_utils.to_categorical(y_prediction, nb_classes) #If I want to do SparseCategoricalAccuracy

#   #Create confusion matrix and normalizes it over predicted (columns)
#   # result = confusion_matrix(y_test, y_prediction , normalize='pred')  #if I want percentages instead of raw counts

  
#   cm = confusion_matrix(y_test, y_prediction)
#   cm = pd.DataFrame(cm, range(10),range(10))

#   #This shows a pretty confusion matrix which I don't neeed to show right now
#   # plt.figure(figsize = (10,10))
#   # sns.heatmap(cm, annot=True, annot_kws={"size": 12}) # font size
#   # plt.show()
#   # cm_normal = cm

#   return cm


### Weighted Categorical Cross Entropy Class

In [5]:
class WeightedCategoricalCrossentropy(tf.keras.losses.CategoricalCrossentropy):

  def __init__(self, cost_mat, name='weighted_categorical_crossentropy', **kwargs):

    cost_mat = np.array(cost_mat)   
    ## when loading from config, self.cost_mat returns as a list, rather than an numpy array. 
    ## Adding the above line fixes this issue, enabling .ndim to call sucessfully. 
    ## However, this is probably not the best implementation
    assert(cost_mat.ndim == 2)
    assert(cost_mat.shape[0] == cost_mat.shape[1])
    super().__init__(name=name, **kwargs)
    self.cost_mat = K.cast_to_floatx(cost_mat)

  def __call__(self, y_true, y_pred, sample_weight=None):
    assert sample_weight is None, "should only be derived from the cost matrix"  
    return super().__call__(
        y_true=y_true, 
        y_pred=y_pred, 
        sample_weight=get_sample_weights(y_true, y_pred, self.cost_mat),
    )


  def get_config(self):
    config = super().get_config().copy()
    # Calling .update on the line above, during assignment, causes an error with config becoming None-type.
    config.update({'cost_mat': (self.cost_mat)})
    return config

  @classmethod
  def from_config(cls, config):
    # something goes wrong here and changes self.cost_mat to a list variable.
    # See above for temporary fix
    return cls(**config)

def get_sample_weights(y_true, y_pred, cost_m):
    num_classes = len(cost_m)

    y_pred.shape.assert_has_rank(2)
    assert(y_pred.shape[1] == num_classes)
    y_pred.shape.assert_is_compatible_with(y_true.shape)

    y_pred = K.one_hot(K.argmax(y_pred), num_classes)

    y_true_nk1 = K.expand_dims(y_true, 2)
    y_pred_n1k = K.expand_dims(y_pred, 1)
    cost_m_1kk = K.expand_dims(cost_m, 0)

    sample_weights_nkk = cost_m_1kk * y_true_nk1 * y_pred_n1k
    sample_weights_n = K.sum(sample_weights_nkk, axis=[1, 2])

    return sample_weights_n


# Register the loss in the Keras namespace to enable loading of the custom object.
tf.keras.losses.WeightedCategoricalCrossentropy = WeightedCategoricalCrossentropy
 

### WeightedCategoricalCross Entropy Function 

In [6]:
def PA_method(cost_matrix):

  model3 = Sequential()
  model3.add(Dense(512, input_shape=(784,), kernel_initializer=tf.keras.initializers.glorot_uniform(seed=42)))
  model3.add(Activation('relu'))
  model3.add(Dropout(0.2))
  model3.add(Dense(512, kernel_initializer=tf.keras.initializers.glorot_uniform(seed=42)))
  model3.add(Activation('relu'))
  model3.add(Dropout(0.2))
  model3.add(Dense(10,kernel_initializer=tf.keras.initializers.glorot_uniform(seed=42)))
  model3.add(Activation('softmax'))

  rms = RMSprop()

  model3.compile(loss=WeightedCategoricalCrossentropy(cost_matrix), optimizer=rms,  metrics='categorical_accuracy',)
  callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

  model3_history = model3.fit(X_train, Y_train,
            batch_size=batch_size, epochs=nb_epoch, verbose=2,
            validation_data=(X_test, Y_test), shuffle=False, use_multiprocessing=True
            ,callbacks = [callback]
            )

 

  #Predict
  y_prediction = model3.predict(X_test)
  y_prediction  = np.argmax(y_prediction, axis=1)
  # Y_prediction = np_utils.to_categorical(y_prediction, nb_classes)

  #Create confusion matrix and normalizes it over predicted (columns)
  # result = confusion_matrix(y_test, y_prediction , normalize='pred')

  

  cm3 = confusion_matrix(y_test, y_prediction)
  cm3 = pd.DataFrame(cm3, range(10),range(10))
  # plt.figure(figsize = (10,10))
  # cm3
  # sns.heatmap(cm2, annot=True, annot_kws={"size": 12}) # font size
  # plt.show()

  # cm_using_weighted_new = cm3

  return cm3

### *Keep Track of Experimental Admin Stuff - #Runs and #CostMatrix

> Change the cost matrix and number of runs and check the file extension name 


#### *Define Cost Matrix and Method

In [7]:
# Experimental Admin Stuff
cost_matrix = np.ones((10,10))

### Weight of Misclassification
## ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
cost_matrix[7, 9] = 100
cost_str = str(cost_matrix[7, 9])
## ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

from datetime import date
today = date.today()
file_date = today.strftime("%Y_%m_%d")

### File Extension to reference in JMP : weights_method_cost
## ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
file_extension = "w[7,9]_PA_" + cost_str
## ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~


np.set_printoptions(suppress=True)
np.set_printoptions(precision=2)

print("Last run using ", cost_str)

Last run using  100.0


#### Run Experiments

In [8]:
cm = np.zeros([10,10])
combined_cms = np.empty((1,100))

## Define the total number of runs
### ~~~~~~~~~
runs = 30
### ~~~~~~~~~~

for i in range(0,runs+1):
  print(i)
  cm2 =  PA_method(cost_matrix)    #Individual CM
  print("CM: \n", cm2)
  # cm += cm2                   #Aggregating for an Average
  cm2_array = np.asarray(cm2)  #Indiv CM as array for storing
  combined_cms = np.vstack((combined_cms,cm2_array.reshape((1,100))))

# cm_new = cm/30

0
Epoch 1/15
235/235 - 4s - loss: 0.6031 - categorical_accuracy: 0.8673 - val_loss: 1.1362 - val_categorical_accuracy: 0.9443 - 4s/epoch - 17ms/step
Epoch 2/15
235/235 - 2s - loss: 0.2838 - categorical_accuracy: 0.9424 - val_loss: 0.1574 - val_categorical_accuracy: 0.9491 - 2s/epoch - 6ms/step
Epoch 3/15
235/235 - 1s - loss: 0.2112 - categorical_accuracy: 0.9588 - val_loss: 0.1679 - val_categorical_accuracy: 0.9699 - 1s/epoch - 6ms/step
Epoch 4/15
235/235 - 2s - loss: 0.1606 - categorical_accuracy: 0.9686 - val_loss: 0.1278 - val_categorical_accuracy: 0.9691 - 2s/epoch - 9ms/step
Epoch 5/15
235/235 - 2s - loss: 0.1408 - categorical_accuracy: 0.9749 - val_loss: 0.3324 - val_categorical_accuracy: 0.9721 - 2s/epoch - 7ms/step
Epoch 6/15
235/235 - 2s - loss: 0.1274 - categorical_accuracy: 0.9783 - val_loss: 0.3927 - val_categorical_accuracy: 0.9751 - 2s/epoch - 8ms/step
Epoch 7/15
235/235 - 1s - loss: 0.1182 - categorical_accuracy: 0.9813 - val_loss: 0.1500 - val_categorical_accuracy: 0.97

#### Save the 30/X confusion matrices

In [9]:
#Save 30 confusion matrices

import pickle

str_runs = str(runs)

file_name = str_runs + "CM_" + file_extension + "_" + file_date + "_.pkl"
print(file_name, " will be saved with ", combined_cms.shape)

with open(file_name, 'wb') as file:
      
    # A new file will be created
    pickle.dump(combined_cms, file)




# Open the file in binary mode
with open(file_name, 'rb') as file:
      
    # Call load method to deserialze
    var = pickle.load(file)
  
    print(var)

print(file_name, " was opened with ", var.shape)

from google.colab import files
files.download( file_name )  

print(file_name, " was saved to Downloads ")


30CM_w[7,9]_PA_100.0_2023_02_15_.pkl  will be saved with  (32, 100)
[[  0.   0.   0. ...   0.   0.   0.]
 [968.   1.   1. ...  44.   0. 937.]
 [966.   1.   3. ...  17.   0. 963.]
 ...
 [970.   1.   2. ...   7.   1. 972.]
 [971.   1.   2. ...  40.   0. 925.]
 [973.   1.   0. ...   1.   0. 983.]]
30CM_w[7,9]_PA_100.0_2023_02_15_.pkl  was opened with  (32, 100)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

30CM_w[7,9]_PA_100.0_2023_02_15_.pkl  was saved to Downloads 


In [10]:
# How many categories are there in the test set?

truth_num_per_category = Y_test.sum(axis=0)
print(truth_num_per_category)

[ 980. 1135. 1032. 1010.  982.  892.  958. 1028.  974. 1009.]


# Analyze 

I am now going to load the Average CM and try to get it in a format where I can make it a 1x100 and load all 30 CMs so that we can visualize their distributions in a a big histogram_matrix. At this point the Google Colab variables are gone and I have to reoad them 

In [11]:
# I need to remove the first placeholder row of zeros
combined_cms = combined_cms[1:(runs+1)]

In [12]:
import numpy as np
import pandas as pd
empty_cm = np.zeros((10,10))  
empty_cm=pd.DataFrame(empty_cm)

empty_cm.columns = ['0p', '1p', '2p', '3p', '4p', '5p', '6p', '7p', '8p', '9p']
empty_cm.index = ['0t', '1t', '2t', '3t', '4t', '5t', '6t', '7t', '8t', '9t']

# print(myvar_cm_average)

empty_cm_array = np.asarray(empty_cm)
empty_cm_array_1_100 = np.reshape(empty_cm_array,(1,100))
# print(cm_average_array)

df = empty_cm
df_new = pd.DataFrame(empty_cm_array_1_100,  columns=pd.MultiIndex.from_product([ df.index,df.columns]))
df_new.columns.to_flat_index()
df_new.columns   = ['_'.join(col) for col in df_new.columns.values]

# Now convert combined_cms of size 30x100 to a panda dataframe
combined_cms_df = pd.DataFrame(combined_cms, columns=[df_new.columns])

combined_cms_df

,0t_0p,0t_1p,0t_2p,0t_3p,0t_4p,0t_5p,0t_6p,0t_7p,0t_8p,0t_9p,...,9t_0p,9t_1p,9t_2p,9t_3p,9t_4p,9t_5p,9t_6p,9t_7p,9t_8p,9t_9p
0,968.0,1.0,1.0,1.0,1.0,1.0,2.0,0.0,4.0,1.0,...,2.0,3.0,0.0,3.0,18.0,2.0,0.0,44.0,0.0,937.0
1,966.0,1.0,3.0,0.0,2.0,3.0,1.0,0.0,2.0,2.0,...,1.0,2.0,0.0,4.0,19.0,3.0,0.0,17.0,0.0,963.0
2,971.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,2.0,1.0,...,3.0,3.0,0.0,5.0,15.0,1.0,1.0,9.0,0.0,972.0
3,971.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,2.0,1.0,...,4.0,4.0,0.0,5.0,18.0,1.0,0.0,9.0,0.0,968.0
4,973.0,1.0,0.0,0.0,2.0,0.0,1.0,0.0,2.0,1.0,...,3.0,2.0,0.0,3.0,15.0,3.0,1.0,2.0,2.0,978.0
5,971.0,1.0,1.0,0.0,2.0,0.0,1.0,1.0,2.0,1.0,...,3.0,3.0,0.0,2.0,20.0,3.0,0.0,6.0,0.0,972.0
6,973.0,1.0,0.0,1.0,0.0,1.0,2.0,0.0,1.0,1.0,...,1.0,2.0,0.0,2.0,14.0,1.0,0.0,8.0,0.0,981.0
7,972.0,1.0,1.0,0.0,1.0,0.0,2.0,1.0,2.0,0.0,...,3.0,3.0,0.0,3.0,15.0,2.0,0.0,5.0,0.0,978.0
8,971.0,1.0,1.0,0.0,2.0,2.0,1.0,0.0,1.0,1.0,...,3.0,4.0,0.0,5.0,18.0,2.0,0.0,7.0,0.0,970.0
9,970.0,1.0,1.0,0.0,2.0,0.0,2.0,0.0,3.0,1.0,...,2.0,4.0,0.0,6.0,16.0,0.0,0.0,8.0,0.0,973.0


In [13]:
combined_cms_df["7t_9p"]

,7t_9p
0,4.0
1,8.0
2,9.0
3,16.0
4,28.0
5,11.0
6,13.0
7,15.0
8,16.0
9,16.0


In [14]:
np.average(combined_cms_df["7t_9p"])

12.266666666666667

In [15]:
csv_filename = file_name[:-4] + ".csv"

combined_cms_df.to_csv(csv_filename)


from google.colab import files
files.download(csv_filename )

print("Downloading ", csv_filename , " of shape ", combined_cms_df.shape)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [16]:
df = pd.DataFrame(sum(var)/len(var), columns=["Values"]) 
# print(df)

df.style.format({
  'Values': lambda val: f'{val:,.2f}',
})

(df.sort_values(by="Values", ascending=False)[0:20])


df_sorted = df.sort_values(by="Values", ascending=False)[10:]  #the top 10 are usually diagonal


df_sorted.style.format({
  'Values': lambda val: f'{val:,.2f}',
})

print("On average...")
print("Num 1 misclassifications are misclassifying a 9 as a 4 (",(df_sorted["Values"].index[0]), ") ", (df_sorted["Values"].values[0]), " times" )
print("Num 2 misclassifications are misclassifying a 6 as a 5 (", (df_sorted["Values"].index[1]), ") ", (df_sorted["Values"].values[1]), " times" )
print("Num 3 misclassifications are misclassifying a 7 as a 9 (",(df_sorted["Values"].index[2]), ") ", (df_sorted["Values"].values[2]), " times" )




On average...
Num 1 misclassifications are misclassifying a 9 as a 4 ( 55 )  844.1875  times
Num 2 misclassifications are misclassifying a 6 as a 5 ( 94 )  16.875  times
Num 3 misclassifications are misclassifying a 7 as a 9 ( 65 )  16.71875  times


In [17]:
df_percents = pd.DataFrame( ((sum(var)*100/len(var)).reshape((10,10))/truth_num_per_category).reshape((100)), columns = ["Values"])


df_sorted_percents = df_percents.sort_values(by="Values", ascending=False)[10:]  #the top 10 are usually diagonal

df_sorted_percents.style.format({
  'Values': lambda val: f'{val:,.2f}',
})

print("On average .. ")
print("Num 1 percent misclassifications are 6 as a 5  (",(df_sorted_percents["Values"].index[0]), ") ", (df_sorted_percents["Values"].values[0]), " percent" )
print("Num 2 percent misclassifications are 9 as a 4 (", (df_sorted_percents["Values"].index[1]), ") ", (df_sorted_percents["Values"].values[1]), " percent" )
print("Num 3 percent misclassifications are 7 as a 9 (",(df_sorted_percents["Values"].index[2]), ") ", (df_sorted_percents["Values"].values[2]), " percent" )






On average .. 
Num 1 percent misclassifications are 6 as a 5  ( 77 )  92.55532587548637  percent
Num 2 percent misclassifications are 9 as a 4 ( 65 )  1.87429932735426  percent
Num 3 percent misclassifications are 7 as a 9 ( 94 )  1.7184317718940938  percent


# Extraneous

To reference later: 

https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/structured_data/imbalanced_data.ipynb#scrollTo=UJ589fn8ST3x

To train a model with class weights:

```
class_weight = {0: weight_for_0, 1: weight_for_1}

weighted_model = make_model()
weighted_model.load_weights(initial_weights)

weighted_history = weighted_model.fit(
    train_features,
    train_labels,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[early_stopping],
    validation_data=(val_features, val_labels),

    # The class weights go here
    class_weight=class_weight)
```



In [18]:
# Extraneous information I am not using at the moment

# model.compile(
#      optimizer='adam',
#      loss=WeightedCategoricalCrossentropy(cost_matrix)
#      )

## Model Saving

# model.save(save_version_dir,save_format='tf')

## Model Loading

# model = tf.keras.models.load_model(
#     save_version_dir,
#     compile=True,
#     custom_objects={
#         'WeightedCategoricalCrossentropy': WeightedCategoricalCrossentropy(cost_matrix)
#         }
#     )
 